# Installs

In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 582.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 35.5 MB/s eta 0:00:00



This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 9.5 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 2.95 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.c

In [ ]:
# '''
# If torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:
# '''

# !pip install torchsummaryx==1.1.0

# Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"llynoirx","key":"b2ce5eadd838f8ca8066ecc979807b17"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!kaggle competitions download -c hw3p2asr-s24

 99% 3.72G/3.74G [00:26<00:00, 169MB/s]
100% 3.74G/3.74G [00:26<00:00, 152MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data
'''
!unzip -q hw3p2asr-s24.zip
!ls

11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data


# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

### Data Analysis (HW3 Quiz)



In [ ]:
# Number of mfccs in train-clean-100 dataset (Q2)
train_mfcc_path = "/content/11-785-s24-hw3p2/train-clean-100/mfcc"
train_mfcc = []

for file in os.listdir(train_mfcc_path):
    mfcc = np.load(os.path.join(train_mfcc_path, file))
    train_mfcc.append(mfcc)

print("Number of train mfccs:", len(train_mfcc)) #28539

Number of train mfccs: 28539


In [ ]:
# You might want to play around with the mapping as a sanity check here

# Map below sequence of phonemes using CMUdict_ARPAabet (Q3)
phoneme_seq = ['B', 'IH', 'K', 'SH', 'AA'] # Q3
mapped_seq = [CMUdict_ARPAbet.get(phoneme, "?") for phoneme in phoneme_sequence]

print(mapped_seq) #['b', 'I', 'k', 'S', 'a']

['b', 'I', 'k', 'S', 'a']


### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, partition):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.transcript_dir = os.path.join(root, partition, "transcript")

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS
        self.mapped_phonemes = {phoneme: idx for idx, phoneme in enumerate(self.PHONEMES)}

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        self.features, self.labels = [], []

        for mfcc_file, transcript_file in zip(self.mfcc_files, self.transcript_files):
          mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_file))
          transcript = np.load(os.path.join(self.transcript_dir, transcript_file))

          # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
          # Cepstral Norm of mfcc
          '''
          You may decide to do this in __getitem__ if you wish.
          However, doing this here will make the __init__ function take the load of
          loading the data, and shift it away from training.
          '''
          mfcc_mean = np.mean(mfcc)
          mfcc_sd = np.std(mfcc)
          cepstral_norm = (mfcc - mfcc_mean) / (mfcc_sd + 1e-5)


          transcript = transcript[1:-1]
          transcript_labels = [self.mapped_phonemes[idx] for idx in transcript]

          self.features.append(mfcc)
          self.labels.append(transcript_labels)

    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[ind]))

        transcript = np.load(os.path.join(self.transcript_dir, self.transcript_files[ind]))
        transcript = transcript[1:-1]
        labels = [self.mapped_phonemes[i] for i in transcript]

        # Convert the data into PyTorch tensors
        return torch.tensor(mfcc), torch.tensor(labels).long()


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [m for m,t in batch]
        # batch of output phonemes
        batch_transcript = [t for m,t in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        lengths_mfcc = [len(m) for m in batch_mfcc] # TODO
        batch_mfcc = pad_sequence(batch_mfcc, batch_first=True, padding_value=0) # TODO

        lengths_transcript = [len(t) for t in batch_transcript] # TODO
        batch_transcript =  pad_sequence(batch_transcript, batch_first=True, padding_value=0) # TODO
        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        # Time Masking
        time_masking = tat.TimeMasking(time_mask_param=20)
        batch_mfcc = time_masking(batch_mfcc)

        # Frequency Masking
        freq_masking = tat.FrequencyMasking(freq_mask_param=7)
        batch_mfcc = freq_masking(batch_mfcc)

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc, batch_transcript, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)


### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):

    def __init__(self, root, partition):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, partition, "mfcc")
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.PHONEMES = PHONEMES
        self.length = len(self.mfcc_files)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        self.features = []

        for mfcc_file in self.mfcc_files:
          mfcc = np.load(os.path.join(self.mfcc_dir, mfcc_file))

          # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
          # Cepstral Norm of mfcc
          '''
          You may decide to do this in __getitem__ if you wish.
          However, doing this here will make the __init__ function take the load of
          loading the data, and shift it away from training.
          '''
          mfcc_mean = np.mean(mfcc)
          mfcc_sd = np.std(mfcc)
          cepstral_norm = (mfcc - mfcc_mean) / (mfcc_sd + 1e-5)

          self.features.append(mfcc)

    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[ind]))
        return torch.tensor(mfcc)


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [m for m  in batch]

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        lengths_mfcc = [len(m) for m in batch_mfcc] # TODO
        batch_mfcc = pad_sequence(batch_mfcc, batch_first=True, padding_value=0) # TODO

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc, torch.tensor(lengths_mfcc)



### Config - Hyperparameters

In [962]:
root = '/content/11-785-s24-hw3p2/'

# Feel free to add more items here
config = {
    "beam_width" : 2,
    "lr"         : 2e-3,
    "epochs"     : 50,
    "batch_size" : 64  # Increase if your device can handle it
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [963]:
# get me RAMMM!!!!
import gc
gc.collect()

34

In [964]:
# Create objects for the dataset class
train_data = AudioDataset(root=root, partition="train-clean-100")
val_data = AudioDataset(root=root,partition="dev-clean") # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(root=root, partition="test-clean") #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(train_data,
                                           batch_size=config['batch_size'],
                                           shuffle=True,
                                           collate_fn=train_data.collate_fn)
val_loader = torch.utils.data.DataLoader(val_data,
                                         batch_size=config['batch_size'],
                                         shuffle=False,
                                         collate_fn=val_data.collate_fn)
test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size=config['batch_size'],
                                          shuffle=False,
                                          collate_fn = test_data.collate_fn)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [965]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1693, 27]) torch.Size([64, 208]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?

        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = __, hidden_size = 256, num_layers = 1)

        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
        )


        self.logSoftmax = #TODO: Apply a log softmax here. Which dimension would apply it on ?

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        pass

## Initialize Basic Network
(If trying out the basic Network)

In [ ]:
torch.cuda.empty_cache()

model = Network().to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

NameError: name 'Network' is not defined

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [966]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [967]:
from pickle import TRUE
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = torch.nn.LSTM(input_size=2*input_size,
                                   hidden_size=hidden_size,
                                   num_layers=1,
                                   bias=True,
                                   batch_first=True,
                                   bidirectional=True) # TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence
        x_padded, x_padded_len = pad_packed_sequence(x_packed, batch_first=True)

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        x_padded, x_padded_len = self.trunc_reshape(x_padded, x_padded_len)

        # TODO: Pack Padded Sequence. What output(s) would you get?
        x_packed = pack_padded_sequence(x_padded, x_padded_len, batch_first=True, enforce_sorted=False)

        # TODO: Pass the sequence through bLSTM
        y_packed, (hidden, cell) = self.blstm(x_packed)

        # What do you return?
        return y_packed

    def trunc_reshape(self, x, x_lens):

        batch_size, seq_len, features = x.size()

        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        if seq_len%2 != 0:
          # x = x[:, :torch.div(input=seq_len, other=2, rounding_mode='floor')*2, :]
          x = x[:, :-1, :]
          x_lens = x_lens - 1

        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        x = x.reshape(batch_size, seq_len//2, features*2)

        # TODO: Reduce lengths by the same downsampling factor
        x_lens = torch.div(input=x_lens, other=2, rounding_mode='floor')

        return x, x_lens

### Encoder

In [968]:
class PrintShape(nn.Module):
    def __init__(self, message):
        super(PrintShape, self).__init__()
        self.message = message

    def forward(self, x):
        print(f"{self.message} {str(x.shape)}")
        return x

In [969]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''

    def __init__(self, input_size, encoder_hidden_size=64):
        super(Encoder, self).__init__()

        print(x.shape)
        self.embedding = torch.nn.Sequential(
            # PrintShape(message="before anything"),
            PermuteBlock(),
            # PrintShape(message="1st permute"),
            torch.nn.Conv1d(input_size, encoder_hidden_size, kernel_size=3, padding=1),
            torch.nn.BatchNorm1d(encoder_hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),

            torch.nn.Conv1d(encoder_hidden_size, encoder_hidden_size*2, kernel_size=3, padding=1),
            torch.nn.BatchNorm1d(encoder_hidden_size*2),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),

            torch.nn.Conv1d(encoder_hidden_size*2, encoder_hidden_size*4, kernel_size=3, padding=1),
            torch.nn.BatchNorm1d(encoder_hidden_size*4),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),

            PermuteBlock(),
            # PrintShape(message="after ending permute (embedding done)")
        )


        # self.linear = torch.nn.Conv1d(encoder_hidden_size, encoder_hidden_size, kernel_size=3)

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            pBLSTM(input_size=4*encoder_hidden_size, hidden_size=encoder_hidden_size))
            # pBLSTM(input_size=2*encoder_hidden_size, hidden_size=encoder_hidden_size))

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        x_embedded = self.embedding(x)
        # print("x embedded shape", x_embedded.shape)
        # x_transformed = self.linear(x_embedded)

        # TODO: Pack Padded Sequence
        x_packed = pack_padded_sequence(x_embedded, x_lens, batch_first=True, enforce_sorted=False)

        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        y = self.pBLSTMs(x_packed)

        # TODO: Pad Packed Sequence
        encoder_outputs, encoder_lens = pad_packed_sequence(y, batch_first=True)

        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

### Decoder

In [970]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size=41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(),
            torch.nn.BatchNorm1d(2*embed_size),
            PermuteBlock(),
            # TODO define your MLP arch. Refer HW1P2
            # Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(2*embed_size, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, output_size)
        )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        mlp_out = self.mlp(encoder_out)

        #TODO Think what should be the final output of the decoder for the classification
        out = self.softmax(mlp_out)
        return out

In [971]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size=len(PHONEMES), encoder_hidden_size=64):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
            PermuteBlock(),
            tat.TimeMasking(time_mask_param=70),
            tat.FrequencyMasking(freq_mask_param=7),
            PermuteBlock()
        )
        self.encoder        = Encoder(input_size, encoder_hidden_size)
        self.decoder        = Decoder(embed_size, output_size)


    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [972]:
model = ASRModel(
    input_size  = 27, #from handout
    embed_size  = 64,
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

torch.Size([64, 1693, 27])
ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): TimeMasking()
    (2): FrequencyMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(27, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): Dropout(p=0.2, inplace=False)
      (5): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): ReLU()
      (8): Dropout(p=0.2, inplace=False)
      (9): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (11): ReLU()
      (12): Dropout(p=0.2, inplace=False)
      (13): PermuteBlock()
    )
    (pBLSTMs): Sequential(
      (0): pBLSTM(
  

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations.PermuteBlock_0,-,"[64, 27, 1693]",NaN,NaN
1_augmentations.TimeMasking_1,-,"[64, 27, 1693]",NaN,NaN
2_augmentations.FrequencyMasking_2,-,"[64, 27, 1693]",NaN,NaN
3_augmentations.PermuteBlock_3,-,"[64, 1693, 27]",NaN,NaN
4_encoder.embedding.PermuteBlock_0,-,"[64, 27, 1693]",NaN,NaN
5_encoder.embedding.Conv1d_1,"[27, 64, 3]","[64, 64, 1693]",5248.0,8776512.0
6_encoder.embedding.BatchNorm1d_2,[64],"[64, 64, 1693]",128.0,64.0
7_encoder.embedding.ReLU_3,-,"[64, 64, 1693]",NaN,NaN
8_encoder.embedding.Dropout_4,-,"[64, 64, 1693]",NaN,NaN


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [973]:
#TODO

criterion = torch.nn.CTCLoss(reduction='mean') # Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr=config["lr"]) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=PHONEMES, log_probs_input=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.01, patience=2)
# scheduler = CosineAnnealingLR(optimizer, T_max=config['epochs'], eta_min=0.001)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [976]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP=LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    decoded, scores, timesteps, out_seq_len = decoder.decode(torch.permute(output, (1, 0, 2)), seq_lens=output_lens)
    # decoded, scores, timesteps, out_seq_len = decoder.decode(output, seq_lens=output_lens)

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred_seq = "".join([PHONEME_MAP[p] for p in decoded[i][0][:out_seq_len[i][0]]])
        pred_strings.append(pred_seq)

    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP=LABELS): #

    dist            = 0
    batch_size      = label.shape[0]
    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        label_slice = label[i][:label_lens[i]]
        label_string = "".join([PHONEME_MAP[p.item()] for p in label_slice])
        dist += Levenshtein.distance(pred_strings[i], label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

# Test Implementation

In [977]:
# test code to check shapes
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)

    loss = criterion(h, y, lh, ly)
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 1468, 41])
torch.Size([1468, 64, 41]) torch.Size([64, 265])
273.859375


# WandB

You will need to fetch your api key from wandb.ai

In [978]:
import wandb
wandb.login(key="f3b2c9738d42f0a8cccb942fd7050e6854a48f84")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [979]:
run = wandb.init(
    name = "kh-early-submission2", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

lr,█▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▃▁▁▁▁▁▁▁█▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
valid_dist,████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,▄▁▁▁▁▁▁▁█▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
lr,0.0
train_loss,1.09955
valid_dist,23.53205
valid_loss,1.06469


# Train Functions

In [980]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):



    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)
            h = torch.permute(h, (1, 0, 2))

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [981]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
# !mkdir models

In [982]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "./models/epoch_model.pth" #TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "./models/best_model.pth"

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr']) #TODO

    train_loss              = train_model(model, train_loader, criterion, optimizer) #TODO
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder) #TODO
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/50


	Train Loss 2.8725	 Learning Rate 0.0020000
	Val Dist 46.8666%	 Val Loss 2.2940
Saved epoch model
Saved best model

Epoch: 2/50


	Train Loss 1.8014	 Learning Rate 0.0020000
	Val Dist 36.0307%	 Val Loss 1.7342
Saved epoch model
Saved best model

Epoch: 3/50


	Train Loss 1.5292	 Learning Rate 0.0020000
	Val Dist 31.8273%	 Val Loss 1.4160
Saved epoch model
Saved best model

Epoch: 4/50


	Train Loss 1.4537	 Learning Rate 0.0020000
	Val Dist 29.6344%	 Val Loss 1.3236
Saved epoch model
Saved best model

Epoch: 5/50


	Train Loss 1.3468	 Learning Rate 0.0020000
	Val Dist 28.1575%	 Val Loss 1.2547
Saved epoch model
Saved best model

Epoch: 6/50


	Train Loss 1.3173	 Learning Rate 0.0020000
	Val Dist 30.2319%	 Val Loss 1.3532
Saved epoch model

Epoch: 7/50


	Train Loss 1.2582	 Learning Rate 0.0020000
	Val Dist 25.1151%	 Val Loss 1.1401
Saved epoch model
Saved best model

Epoch: 8/50


Val:  74%|███████▍  | 32/43 [02:57<00:51,  4.64s/it, dist=27.7046, loss=1.3852]

# Generate Predictions and Submit to Kaggle

In [959]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 10 #TODO

test_decoder    = test_decoder = CTCBeamDecoder(labels=LABELS,
                                                beam_width=TEST_BEAM_WIDTH,
                                                blank_id=0,
                                                num_processes=torch.multiprocessing.cpu_count()
                                                ) #TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    # x = torch.permute(x, (1, 0, 2))
    with torch.no_grad():
        h, lh = model(x, lx)

    beam_results, beam_scores, timesteps, out_lens = test_decoder.decode(h, lh)
    predict_string= decode_prediction(h, lh, test_decoder, LABELS) # TODO call decode_prediction
    #TODO save the output in results array.
    results.extend(predict_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

Testing


100%|██████████| 41/41 [00:48<00:00,  1.19s/it]


In [960]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

In [961]:
!kaggle competitions submit -c hw3p2asr-s24slack -f submission.csv -m "I made it!"

100% 957k/957k [00:00<00:00, 1.41MB/s]
Successfully submitted to HW3P2_ASR-S24(slack)